In [9]:
import json
import numpy as np

In [10]:
with open("data.json", "r", encoding="utf8") as f:
    js = json.load(f)

# Обучающая выборка (все кроме 10 последних)
spam_msg_ex = [row["text_msg"] for row in js["spam"][:-10]]
nospam_msg_ex = [row["text_msg"] for row in js["nospam"][:-10]]

# Тестовая выборка (10 последних)
spam_msg_test = [row["text_msg"] for row in js["spam"][-10:]]
nospam_msg_test = [row["text_msg"] for row in js["nospam"][-10:]]

# Создаем словарь, который содержит слово и сколько раз оно встречалось в наборе данных:
# пример spam = {'привет': 4, ...}, nospam = {'привет': 29, ...}
from collections import Counter 
def create_dict_from_array(array_):
    words = [word.lower() for row in array_ for word in row.split()]
    return Counter(words)

spam = create_dict_from_array(spam_msg_ex)
nospam = create_dict_from_array(nospam_msg_ex)

In [11]:
def test_spam_v2(text, w=0.55, debug=False):
    words = [word.lower() for word in text.split()]
    P = []
    for word in words:
        count = spam[word] + nospam[word]
        p = spam[word]/(count) if count != 0 else 0.5    
        eps = 0.001
        p = p - eps if p > 0.5 else p+eps # if p!= 0.5 else 0.5
        P.append(p)
    
    PS = np.prod(P)
    PH = np.prod([1-p for p in P])
    rez_P = PS/(PS+PH) if PS+PH != 0 else 0
    if debug:
        print(P)
        print("P(B|A):", PS, "\nP(B|no A):", PH, "\nP(A|B):", rez_P)
        return float(rez_P)
    return  True if rez_P > w else False

In [12]:
def my_contact(email):
    """
    1) Реализовать функцию my_contact, которая возвращает true если email находится в контактах (те сообщения, которые не спам явно от контактов),
    2) Реализовать возможность добавлять новых контактов
    3) Сохранить список контактов как json файл
    """
    return False

In [13]:
def is_spam_msg(msg):
    email = msg["email"]
    text = msg["text_msg"]
    if my_contact(email):
        return False
    return test_spam_v2(text)

In [18]:
is_spam_msg({"email": "elena.ivanova@company.com", "text_msg": "Привет! делаю ремонт"})
#                           ^ email уже встречался как не спам

True